In [16]:
pip install clawpack

In [17]:
import numpy as np
import matplotlib.pyplot as plt
from clawpack import riemann, pyclaw
import sys
import os
from google.colab import drive
from sklearn.decomposition import PCA

# Mount Google Drive if you haven't already
drive.mount('/content/drive')

# Assuming your project is in Google Drive
# Adjust this path to point to your project's root directory
project_path = '/content/drive/MyDrive/cs234_project'  # Change this to your actual path
sys.path.append(project_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 1. Mathematical Definition of the PDE

The 2D advection equation describes the transport of a scalar quantity $q(x,y,t)$ by a velocity field $(u_x, u_y)$:

$$\frac{\partial q}{\partial t} + u_x \frac{\partial q}{\partial x} + u_y \frac{\partial q}{\partial y} = 0$$

Our specific parameters are:
- $u_x = 0.5$ (x-velocity)
- $u_y = 1.0$ (y-velocity)
- Domain: $\Omega = [0,1] \times [0,1]$
- Boundary conditions: Periodic in both x and y directions
- Initial condition: A square pulse with $q = 1.0$ in $[0.1,0.6] \times [0.1,0.6]$ and $q = 0.1$ elsewhere
- Grid: $50 \times 50$ points
- Time domain: $t \in [0, 2.0]$
- Number of time steps: 20

## 2. Solution Representation

### 3.1 Numerical Solution

After solving the PDE, we obtain the solution at each grid point and time step. We organize this solution data into a matrix $Y$ with the following structure:

$$Y =
\begin{bmatrix}
y(x_1, y_1, t_1) & y(x_1, y_1, t_2) & \cdots & y(x_1, y_1, t_m) \\
y(x_2, y_1, t_1) & y(x_2, y_1, t_2) & \cdots & y(x_2, y_1, t_m) \\
\vdots & \vdots & \ddots & \vdots \\
y(x_n, y_p, t_1) & y(x_n, y_p, t_2) & \cdots & y(x_n, y_p, t_m)
\end{bmatrix}$$

- Each row corresponds to a specific spatial location $(x_i, y_j)$
- Each column corresponds to a time step $t_k$
- The matrix has dimensions $(n \cdot p) \times m$ where:
  - $n = 50$ (number of x grid points)
  - $p = 50$ (number of y grid points)
  - $m = 21$ (number of time steps, including initial condition)


In [18]:
from pde.AdvectionEquation import Advection2D, Adv2dModelConfig
config = Adv2dModelConfig()
adv_obj = Advection2D(config)
# Generate initial condition
init_cond = adv_obj.initial_condition()
# Solve PDE
results = adv_obj.step(init_cond)
# Get actual dimensions
n = adv_obj.nx  # number of x grid points
p = adv_obj.ny  # number of y grid points
actual_frames = len(results)  # actual number of frames returned
print(f"Grid dimensions: {n}x{p}")
print(f"Number of frames returned by solver: {actual_frames}")
# Total time steps including initial condition
m = actual_frames + 1
# Create the Y matrix with shape (n*p, m)
Y = np.zeros((n * p, m))
# Fill in the initial condition (t=0)
Y[:, 0] = init_cond.reshape(-1)
# Fill in the remaining time steps
for t in range(1, m):
    # Get the solution at time step t-1 (frames are 0-indexed)
    q_t = results[t-1]
        # Flatten the 2D spatial grid into a column
    Y[:, t] = q_t.reshape(-1)

print(f"\nY matrix shape: {Y.shape}")
# Verify a few values from the Y matrix
print(f"\nValue at (x=10, y=10, t=0): {Y[10 + 10*n, 0]}")
middle_t = m // 2
print(f"Value at (x=25, y=25, t={middle_t}): {Y[25 + 25*n, middle_t]}")
print(Y)

Grid dimensions: 50x50
Number of frames returned by solver: 20

Y matrix shape: (2500, 21)

Value at (x=10, y=10, t=0): 1.0
Value at (x=25, y=25, t=10): 0.11861701264061458
[[0.1        0.1        0.1        ... 0.69898093 0.19590718 0.10000336]
 [0.1        0.1        0.1        ... 0.69909167 0.21808002 0.10004222]
 [0.1        0.1        0.1        ... 0.69910662 0.2284222  0.10034163]
 ...
 [0.1        0.1        0.1        ... 0.48507539 0.10268632 0.1       ]
 [0.1        0.1        0.1        ... 0.49280699 0.1108123  0.1       ]
 [0.1        0.1        0.1        ... 0.49484006 0.12514848 0.10000006]]


The covariance function  is defined in Equation (20) of the paper as:



# Covariance Function and Discretization

## Continuous Covariance Function R(x, ζ)

The covariance function R(x, ζ) is defined in Equation (20) of the paper as:

$$\int_{\Omega} R(x, \zeta) \phi_i(\zeta) d\zeta = \lambda_i \phi_i(x)$$

Where:
- R(x, ζ) = ⟨y(x, t)y(ζ, t)⟩ is the spatial two-point correlation function
- Measures how strongly the values of y(x,t) and y(ζ,t) are correlated over time
- φ_i(x) are the eigenfunctions of R(x, ζ), obtained from the Karhunen–Loève Decomposition (KLD)

## Discretization of Covariance Matrix

From Equation (24) of the paper, the temporal correlation is defined as:

$$C_{tk} = \frac{1}{l} \int_{\Omega} y(\zeta, t) y(\zeta, k) d\zeta$$

Where:
- C_{tk} represents the temporal correlation between states at different time steps
- The integral over Ω converts the continuous form into a discrete covariance matrix

### Discrete Covariance Matrix Formulation

The basic discrete form:

$$R_{ij} = \frac{1}{m} \sum_{t=1}^{m} y(x_i, t) y(x_j, t)$$

### Mean-Subtracted Covariance Matrix

To center the data, use mean subtraction:

$$R_{ij} = \frac{1}{m} \sum_{t=1}^{m} \left( y(x_i, t) - \bar{y}(x_i) \right) \left( y(x_j, t) - \bar{y}(x_j) \right)$$



## Source
Appendix A, Equations (20) and (24) in the referenced paper.


In [19]:
from reward.reward import RewardCalculator
# Reshape Y from (n*p, m) to (n, p, m) for RewardCalculator
Y_3d = Y.reshape(n, p, m)
# Create RewardCalculator instance
reward_calc = RewardCalculator(Y_3d)
# Compute covariance matrix
cov_matrix = reward_calc.compute_covariance_matrix()
print(f"Covariance matrix shape: {cov_matrix.shape}")
print(f"Expected shape: ({n*p}, {n*p})")
type(cov_matrix)


Covariance matrix shape: (2500, 2500)
Expected shape: (2500, 2500)


numpy.ndarray

## **3. Solve the Eigenvalue Problem**

decompose the covariance matrix to find dominant spatial patterns using:

$$
R \Phi = \Lambda \Phi
$$

where:
- $\Phi$ contains **eigenvectors** (KLD modes).
- $\Lambda$ is a **diagonal matrix of eigenvalues** (variance captured by each mode).
- Eigenvalues are sorted in **descending order**, ensuring the most important modes are selected.

**Reference:** Appendix A, Equation (20).

---




In [20]:
# 3. Eigenvalue Decomposition
# Now that we've fixed the RewardCalculator class, we can use its methods directly

# Solve eigenvalue problem
eigenvalues, eigenvectors = reward_calc.solve_eigenvalue_problem()
print(f"Number of eigenvalues: {len(eigenvalues)}")
print(f"Top 5 eigenvalues: {eigenvalues[:5]}")



change 3 

Number of eigenvalues: 2500
Top 5 eigenvalues: [88.7095912  67.38376706 53.2815858  47.78793793 26.32981453]


## **3. Select KLD Modes**

The total **energy captured** by the first k modes is given by:

$$
E_k = \frac{\sum_{i=1}^{k} \lambda_i}{\sum_{j=1}^{n} \lambda_j}
$$

We choose k such that:

$$
E_k \geq 0.99
$$

This ensures we retain **99% of the system's variance**.

**Reference:** Appendix A, Equation (25)

In [21]:
# 4. Mode Selection
# Calculate cumulative energy
cumulative_energy = np.cumsum(eigenvalues) / np.sum(eigenvalues)
threshold = 0.99
num_modes = np.searchsorted(cumulative_energy, threshold) + 1
print(f"Number of modes required to capture {threshold*100}% of energy: {num_modes}")
# Select KLD modes
selected_modes = reward_calc.select_KLD_modes(num_modes)
print(f"Selected modes shape: {selected_modes.shape}")



Number of modes required to capture 99.0% of energy: 14
Selected modes shape: (2500, 14)



## **4. Compute the Reward Function**

The **reward function** is based on the **information gain** from sensor placement.

1. **Extract the sensor locations** from the selected KLD modes:

$$
P_m = \text{selected\_modes}[ \text{sensor\_indices}, :]
$$

2. **Compute the reduced observability matrix**:

$$
T_m = P_m^T P_m
$$

3. **Define the reward as the determinant**:

$$
\text{Reward} = \log \det(T_m)
$$

Equation(12)

In [22]:

# 5. Reward Calculation
# Calculate reward for a sample set of sensor positions
example_sensors = [(25, 25), (10, 40), (30, 15)]
flat_indices = [i + j * n for i, j in example_sensors]
reward = reward_calc.compute_reward_function(flat_indices)
print(f"Reward for KLD Method: {reward}")




Reward for KLD Method: 1.1150733081682884e-07


In [23]:

from reward.reward import RewardCalculator
import numpy as np
import matplotlib.pyplot as plt

print(f"PDE solution data shape: {Y_3d.shape}")

# Create a RewardCalculator instance
reward_calc = RewardCalculator(Y_3d)

# Compute KLD using the original approach
print("Computing KLD using original approach...")
cov_matrix = reward_calc.compute_covariance_matrix()
eigenvalues_orig, eigenvectors_orig = reward_calc.solve_eigenvalue_problem()

# Determine number of modes to keep (99% energy)
cumulative_energy_orig = np.cumsum(eigenvalues_orig) / np.sum(eigenvalues_orig)
threshold = 0.99
num_modes_orig = np.searchsorted(cumulative_energy_orig, threshold) + 1
print(f"Number of modes required to capture {threshold*100}% of energy (original): {num_modes_orig}")

# Select KLD modes
selected_modes_orig = reward_calc.select_KLD_modes(num_modes_orig)
print(f"Selected modes shape (original): {selected_modes_orig.shape}")

print("\nComputing KLD using PCA-based approach...")
selected_modes_pca, eigenvalues_pca, num_modes_pca = reward_calc.compute_kld_with_pca(threshold)
print(f"Number of modes required to capture {threshold*100}% of energy (PCA): {num_modes_pca}")
print(f"Selected modes shape (PCA): {selected_modes_pca.shape}")
print("\nComparing top 5 eigenvalues:")
print(f"Original approach: {eigenvalues_orig[:5]}")
print(f"PCA approach: {eigenvalues_pca[:5]}")
rel_diff = np.abs(eigenvalues_orig[:num_modes_orig] - eigenvalues_pca[:num_modes_orig]) / eigenvalues_orig[:num_modes_orig]
print(f"\nMean relative difference in eigenvalues: {np.mean(rel_diff):.6f}")
print("\nComparing rewards for sample sensor positions:")
example_sensors = [(25, 25), (10, 40), (30, 15)]
flat_indices = [i + j * n for i, j in example_sensors]
reward_orig = reward_calc.compute_reward_function(flat_indices)
reward_pca = reward_calc.compute_reward_function_pca(flat_indices)

print(f"Reward for KLD Method (original): {reward_orig}")
print(f"Reward for KLD Method (PCA): {reward_pca}")

PDE solution data shape: (50, 50, 21)
Computing KLD using original approach...
change 3 

Number of modes required to capture 99.0% of energy (original): 14
Selected modes shape (original): (2500, 14)

Computing KLD using PCA-based approach...
Number of modes required to capture 99.0% of energy (PCA): 14
Selected modes shape (PCA): (2500, 14)

Comparing top 5 eigenvalues:
Original approach: [88.7095912  67.38376706 53.2815858  47.78793793 26.32981453]
PCA approach: [88.7095912  67.38376706 53.2815858  47.78793793 26.32981453]

Mean relative difference in eigenvalues: 0.000000

Comparing rewards for sample sensor positions:
Reward for KLD Method (original): 1.1150733081682884e-07
Reward for KLD Method (PCA): 1.1150733081682884e-07


In [24]:
""""
# 6. Find Optimal Single Sensor Location
print("\nFinding optimal single sensor location...")
best_reward = -float('inf')
best_location = None

# Sample grid points to reduce computation (every 5th point)
step = 5
grid_samples = [(i, j) for i in range(0, n, step) for j in range(0, p, step)]
print(f"Testing {len(grid_samples)} locations...")

for i, j in grid_samples:
    flat_idx = i + j * n
    current_reward = reward_calc.compute_reward_function([flat_idx])

    if current_reward > best_reward:
        best_reward = current_reward
        best_location = (i, j)

print(f"Best single sensor location: {best_location}")
print(f"Reward value: {best_reward}")"""

'"\n# 6. Find Optimal Single Sensor Location\nprint("\nFinding optimal single sensor location...")\nbest_reward = -float(\'inf\')\nbest_location = None\n\n# Sample grid points to reduce computation (every 5th point)\nstep = 5\ngrid_samples = [(i, j) for i in range(0, n, step) for j in range(0, p, step)]\nprint(f"Testing {len(grid_samples)} locations...")\n\nfor i, j in grid_samples:\n    flat_idx = i + j * n\n    current_reward = reward_calc.compute_reward_function([flat_idx])\n\n    if current_reward > best_reward:\n        best_reward = current_reward\n        best_location = (i, j)\n\nprint(f"Best single sensor location: {best_location}")\nprint(f"Reward value: {best_reward}")'

In [25]:
import numpy as np
from GymEnvironment.environment2 import SensorOptimalPlacement


def run_random_policy_benchmark(
        width=50,
        length=50,
        n_sensor=5,
        seed=42,
        num_episodes=10,
        max_horizon = 20,
        N_max = 5,
):
    # Create an instance of your custom env
    env = SensorOptimalPlacement(width=width, length=length, n_sensor=n_sensor, max_horizon= max_horizon, N_max= N_max, seed=seed)

    all_episode_rewards = []

    """
    for episode_idx in range(num_episodes):
        # Reset environment and get initial observation
        obs = env.reset(seed = episode_idx)

        done = False

        while not done:
            # Sample a random action
            action = env.action_space.sample()

            # Take a step in the environment
            obs, reward, done, info = env.step(action)

            # Check for a terminal condition (if your env supports it)
            if done:
                break

        # Store this episode's reward
        # all_episode_rewards.append(episode_reward)
        # print(f"Episode {episode_idx + 1}/{num_episodes} - Reward: {episode_reward}")

    # Print or return some statistics
    avg_reward = np.mean(all_episode_rewards)
    print(f"\nCompleted {num_episodes} episodes using random policy.")
    print(f"Average reward: {avg_reward:.4f}")
    print(f"Reward per episode: {all_episode_rewards}")

    return all_episode_rewards"""
    best_rewards = []
    for episode_idx in range(num_episodes):
        # Reset environment and get initial observation
        obs = env.reset(seed=episode_idx)
        episode_rewards = []
        done = False
        truncated = False
        print(f"Starting episode {episode_idx + 1}/{num_episodes}")
        step = 0
        while not done and not truncated:
            action = env.action_space.sample()
            obs, reward, done, truncated, info = env.step(action)
            episode_rewards.append(reward)
            step += 1
            print(f"  Step {step}, Current reward: {reward:.6f}")
        best_rewards.append(info["max_reward"])
        all_episode_rewards.append(episode_rewards)
        print(f"Episode {episode_idx + 1}/{num_episodes} complete - Max Reward: {info['max_reward']:.6f}")
    print(f"\nCompleted {num_episodes} episodes using random policy.")
    print(f"Best reward overall: {max(best_rewards):.6f}")
    print(f"Average best reward per episode: {np.mean(best_rewards):.6f}")

    return all_episode_rewards, best_rewards


if __name__ == "__main__":
    # Run the benchmark with random actions
    run_random_policy_benchmark()


Starting episode 1/10
  Step 1, Current reward: -32.234410
  Step 2, Current reward: -32.071594
  Step 3, Current reward: -31.869149
  Step 4, Current reward: -29.030027
  Step 5, Current reward: -29.239563
  Step 6, Current reward: -30.483115
  Step 7, Current reward: -28.744890
  Step 8, Current reward: -27.467946
  Step 9, Current reward: -27.055600
  Step 10, Current reward: -26.214720
  Step 11, Current reward: -26.466955
  Step 12, Current reward: -26.612760
  Step 13, Current reward: -26.450141
  Step 14, Current reward: -26.237444
  Step 15, Current reward: -26.121153
  Step 16, Current reward: -26.397831
  Step 17, Current reward: -25.997520
  Step 18, Current reward: -26.636314
  Step 19, Current reward: -26.568679
  Step 20, Current reward: -26.650465
  Step 21, Current reward: -26.868422
Episode 1/10 complete - Max Reward: -25.997520
Starting episode 2/10
  Step 1, Current reward: -28.297287
  Step 2, Current reward: -28.031119
  Step 3, Current reward: -28.560967
  Step 4,